
# Monitoring Performance of Jobs

Optimizing performance from a job and cluster perspective is a critical part of Production Delta Lake Performance. We often use a variety of methods listed in the table below to monitor jobs performance.

|Feature|Use|Link|
|-------|---|----|
|[Ganglia ]Metrics: Cluster Tuning | Controlling the knobs associated with seeking out maximal performance|https://docs.databricks.com/clusters/configure.html, https://www.youtube.com/watch?v=9fa8dnKbfsU|
|Spark UI: DAG Tuning | Controlling the stages associated with a Spark Job and a Query Execution Plan | https://databricks.com/session/understanding-query-plans-and-spark-uis|
|Logs | Spark Driver and Cluster Logs that provide execution and autoscaling details | https://docs.databricks.com/spark/latest/rdd-streaming/debugging-streaming-applications.html|
|EXPLAIN: Physical Plan | physical plan provides the fundamental information about the execution of the query |https://docs.databricks.com/sql/language-manual/sql-ref-syntax-qry-explain.html |

Useful blogs to understand how Delta Lake performance tuning works are listed below.
* [Photon, The Next Generation Query Engine on the Databricks Lakehouse Platform](https://www.databricks.com/blog/2021/06/17/announcing-photon-public-preview-the-next-generation-query-engine-on-the-databricks-lakehouse-platform.html)
* [Faster MERGE Performance With Low-Shuffle MERGE and Photon](https://www.databricks.com/blog/2022/10/17/faster-merge-performance-low-shuffle-merge-and-photon.html)
* [Understanding your Apache Spark Application Through Visualization](https://www.databricks.com/blog/2015/06/22/understanding-your-spark-application-through-visualization.html)


# Cluster Performance

Cluster performance tuning is an important step when quantifying delta performance. The distributed nature of Delta and Spark allow great horizontal scaling by adding more nodes to meet performance SLAs. Generally speaking, leverage autoscaling on Spark clusters to reduce costs and tackle peak throughput workloads.


## Live Metrics - Metrics UI

Databricks cluster performance can be observed in the Ganglia UI (or Metrics UI) which runs live on the cluster. Ganglia metrics are available for Databricks Runtime 12.2 and below and the metrics replacement is now enabled on clusters with DBR 13.

<img src='https://raw.githubusercontent.com/morganmazouchi/Performance-with-Photon/main/Images/Ganglia%20UI.png' width="1500">


# Spark UI

Databricks exposes the Spark UI which will provide a large amount of usable statistics to measure the performance of your jobs. Every job in Spark consists of a series of spark tasks (stages) which form a directed acyclic graph (DAG). Examining these DAGs can help identify bottleneck stages to determine where more performance can be extracted. 

<img src='https://raw.githubusercontent.com/morganmazouchi/Performance-with-Photon/main/Images/Spark%20UI-%20Job%20159.png' width="800">
<img src='https://raw.githubusercontent.com/morganmazouchi/Performance-with-Photon/main/Images/Spark%20UI-%20Job%20166%20-%20No%20Photon.png' width="1300">


## Speed up queries by identifying execution bottlenecks in Query Plans
A common methodology for speeding up queries is to first identify the longest running query operators. We are more interested in total time spent on a task rather than the exact “wall clock time” of an operator as we’re dealing with a distributed system and operators can be executed in parallel. Each query operator comes with a slew of statistics. In the case of a scan operator, metrics include number of files or data read, time spent waiting for cloud storage or time spent reading files. As a result, it is easy to answer questions such as which table should be optimized or whether a join could be improved. All blue DAGs in the query plan confirms that photon was disabled when the query ran. 

<img src='https://raw.githubusercontent.com/morganmazouchi/Performance-with-Photon/main/Images/Databricks%20Shell%20-%20Details%20for%20Query%20299.png' width="1200">

In [0]:
%run ./00-setup $mode="reuse"


username: mojgan.mazouchi@databricks.com
userhome: dbfs:/user/mojgan.mazouchi@databricks.com/PhotonPerformance
database: PhotonPerformance_mojgan_mazouchi_databricks_com_db


In [0]:
spark.conf.set("spark.databricks.photon.enabled", "false")
spark.conf.set("spark.databricks.photon.parquetWriter.enabled", "false")
spark.conf.set("spark.databricks.photon.window.enabled", "false")
spark.conf.set("spark.databricks.photon.sort.enabled", "false")
spark.conf.set("spark.databricks.photon.window.experimental.features.enabled", "false")

In [0]:
%scala
spark.sql("""EXPLAIN SELECT
  T_len.EmpLength,
  T_rate.IntRate,
  count(DISTINCT T.addr_state) cnt_loan_by_state,
  avg(loan_amnt) avg_loan_by_state,
  min(DISTINCT annual_inc) as min_annual_income,
  max(DISTINCT annual_inc) as max_annual_income,
  sum(total_pymnt) totalPayment_by_state
FROM
  LendingClub_silver T
  LEFT JOIN 
  (SELECT row_number() OVER(PARTITION BY addr_state ORDER BY avg_cur_bal DESC) as row_num_avgBal_state, *
  FROM LendingClub_EmpLength) T_len on T_len.emp_length = T.emp_length and T_len.avg_cur_bal BETWEEN 1 AND 2000
  LEFT JOIN LendingClub_IntRate T_rate on T_rate.int_rate = T.int_rate
WHERE
  (annual_inc> 16000) AND loan_status == "Current"
GROUP BY
  1,
  2
HAVING EmpLength IN ('3-5Years', '1year', 'Under1year')""").collect().foreach(println)

[== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[EmpLength#12737, IntRate#12741], functions=[finalmerge_avg(merge sum#12755, count#12756L) AS avg(loan_amnt#12601)#12747, finalmerge_min(merge min#12758) AS min(annual_inc#12612)#12748, finalmerge_max(merge max#12760) AS max(annual_inc#12612)#12749, finalmerge_sum(merge sum#12762) AS sum(cast(total_pymnt#12636 as double))#12750, finalmerge_count(distinct merge count#12752L) AS count(addr_state#12621)#12746L])
 +- Exchange hashpartitioning(EmpLength#12737, IntRate#12741, 200), ENSURE_REQUIREMENTS, [plan_id=4388]
 +- HashAggregate(keys=[EmpLength#12737, IntRate#12741], functions=[merge_avg(merge sum#12755, count#12756L) AS (sum#12755, count#12756L), merge_min(merge min#12758) AS min#12758, merge_max(merge max#12760) AS max#12760, merge_sum(merge sum#12762) AS sum#12762, partial_count(distinct addr_state#12621) AS count#12752L])
 +- HashAggregate(keys=[EmpLength#12737, IntRate#12741, addr_state#12621], functions=[merge_avg(merge sum#12755, count#12756L) AS (sum#12755, count#12756L), merge_min(merge min#12758) AS min#12758, merge_max(merge max#12760) AS max#12760, merge_sum(merge sum#12762) AS sum#12762])
 +- Exchange hashpartitioning(EmpLength#12737, IntRate#12741, addr_state#12621, 200), ENSURE_REQUIREMENTS, [plan_id=4384]
 +- HashAggregate(keys=[EmpLength#12737, IntRate#12741, addr_state#12621], functions=[partial_avg(loan_amnt#12601) AS (sum#12755, count#12756L), partial_min(annual_inc#12612) AS min#12758, partial_max(annual_inc#12612) AS max#12760, partial_sum(cast(total_pymnt#12636 as double)) AS sum#12762])
 +- Project [loan_amnt#12601, annual_inc#12612, addr_state#12621, total_pymnt#12636, EmpLength#12737, IntRate#12741]
 +- BroadcastHashJoin [int_rate#12605], [int_rate#12740], LeftOuter, BuildRight, false
 :- Project [loan_amnt#12601, int_rate#12605, annual_inc#12612, addr_state#12621, total_pymnt#12636, EmpLength#12737]
 : +- BroadcastHashJoin [emp_length#12610], [emp_length#12736], Inner, BuildRight, false
 : :- Project [loan_amnt#12601, int_rate#12605, emp_length#12610, annual_inc#12612, addr_state#12621, total_pymnt#12636]
 : : +- Filter ((((isnotnull(annual_inc#12612) AND (annual_inc#12612 > 16000.0)) AND isnotnull(loan_status#12614)) AND isnotnull(emp_length#12610)) AND (loan_status#12614 = Current))
 : : +- FileScan parquet hive_metastore.photonperformance_mojgan_mazouchi_databricks_com_db.lendingclub_silver[loan_amnt#12601,int_rate#12605,emp_length#12610,annual_inc#12612,loan_status#12614,addr_state#12621,total_pymnt#12636] Batched: true, DataFilters: [isnotnull(annual_inc#12612), (annual_inc#12612 > 16000.0), isnotnull(loan_status#12614), isnotnu..., Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[dbfs:/user/mojgan.mazouchi@databricks.com/PhotonPerformance/lendi..., PartitionFilters: [], PushedFilters: [IsNotNull(annual_inc), GreaterThan(annual_inc,16000.0), IsNotNull(loan_status), IsNotNull(emp_le..., ReadSchema: struct<loan_amnt:float,int_rate:string,emp_length:string,annual_inc:float,loan_status:string,addr...
 : +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=4375]
 : +- Project [emp_length#12736, EmpLength#12737]
 : +- Filter ((((isnotnull(avg_cur_bal#12739) AND (avg_cur_bal#12739 >= 1)) AND (avg_cur_bal#12739 <= 2000)) AND isnotnull(emp_length#12736)) AND EmpLength#12737 IN (3-5Years,1year,Under1year))
 : +- FileScan parquet hive_metastore.photonperformance_mojgan_mazouchi_databricks_com_db.lendingclub_emplength[emp_length#12736,EmpLength#12737,avg_cur_bal#12739] Batched: true, DataFilters: [isnotnull(avg_cur_bal#12739), (avg_cur_bal#12739 >= 1), (avg_cur_bal#12739 <= 2000), isnotnull(e..., Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[dbfs:/user/mojgan.mazouchi@databricks.com/PhotonPerformance/lendi..., PartitionFilters: [], PushedFilters: [IsNotNull(avg_cur_bal), GreaterThanOrEqual(avg_cur_bal,1), LessThanOrEqual(avg_cur_bal,2000), Is..., ReadSchema: struct<emp_length:string,EmpLength:string,av

## Photon execution analysis in Query Plan
If you are using Photon on Databricks clusters, you can view Photon action in the Spark UI. The following screenshot shows the query details DAG. There are two indications of Photon in the DAG. First, Photon operators start with Photon, such as PhotonGroupingAgg. Secondly, in the DAG Photon operators and stages are colored orange, whereas the non-Photon ones are blue.

<img src='https://raw.githubusercontent.com/morganmazouchi/Performance-with-Photon/main/Images/Databricks%20Shell%20-%20Details%20for%20Query%20272.png' width="1800">

In [0]:
#Enable photon and it's support for sort and window functions
spark.conf.set("spark.databricks.photon.enabled", "true")
spark.conf.set("spark.databricks.photon.parquetWriter.enabled", "true")
spark.conf.set("spark.databricks.photon.window.enabled", "true")
spark.conf.set("spark.databricks.photon.sort.enabled", "true")
spark.conf.set("spark.databricks.photon.window.experimental.features.enabled", "true")

In [0]:
%scala
spark.sql("""EXPLAIN SELECT
  T_len.EmpLength,
  T_rate.IntRate,
  count(DISTINCT T.addr_state) cnt_loan_by_state,
  avg(loan_amnt) avg_loan_by_state,
  min(DISTINCT annual_inc) as min_annual_income,
  max(DISTINCT annual_inc) as max_annual_income,
  sum(total_pymnt) totalPayment_by_state
FROM
  LendingClub_silver T
  LEFT JOIN 
  (SELECT row_number() OVER(PARTITION BY addr_state ORDER BY avg_cur_bal DESC) as row_num_avgBal_state, *
  FROM LendingClub_EmpLength) T_len on T_len.emp_length = T.emp_length and T_len.avg_cur_bal BETWEEN 1 AND 100
  LEFT JOIN LendingClub_IntRate T_rate on T_rate.int_rate = T.int_rate
WHERE
  (annual_inc> 16000) AND loan_status == "Current"
GROUP BY
  1,
  2
HAVING EmpLength IN ('3-5Years', '1year', 'Under1year')""").collect().foreach(println)

[== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- ColumnarToRow
 +- PhotonResultStage
 +- PhotonGroupingAgg(keys=[EmpLength#13145, IntRate#13151], functions=[finalmerge_avg(merge sum#13165, count#13166L) AS avg(loan_amnt#13009)#13157, finalmerge_min(merge min#13168) AS min(annual_inc#13020)#13158, finalmerge_max(merge max#13170) AS max(annual_inc#13020)#13159, finalmerge_sum(merge sum#13172) AS sum(cast(total_pymnt#13044 as double))#13160, finalmerge_count(distinct merge count#13162L) AS count(addr_state#13029)#13156L])
 +- PhotonShuffleExchangeSource
 +- PhotonShuffleMapStage
 +- PhotonShuffleExchangeSink hashpartitioning(EmpLength#13145, IntRate#13151, 200)
 +- PhotonGroupingAgg(keys=[EmpLength#13145, IntRate#13151], functions=[merge_avg(merge sum#13165, count#13166L) AS (sum#13165, count#13166L), merge_min(merge min#13168) AS min#13168, merge_max(merge max#13170) AS max#13170, merge_sum(merge sum#13172) AS sum#13172, partial_count(distinct addr_state#13029) AS count#13162L])
 +- PhotonGroupingAgg(keys=[EmpLength#13145, IntRate#13151, addr_state#13029], functions=[merge_avg(merge sum#13165, count#13166L) AS (sum#13165, count#13166L), merge_min(merge min#13168) AS min#13168, merge_max(merge max#13170) AS max#13170, merge_sum(merge sum#13172) AS sum#13172])
 +- PhotonShuffleExchangeSource
 +- PhotonShuffleMapStage
 +- PhotonShuffleExchangeSink hashpartitioning(EmpLength#13145, IntRate#13151, addr_state#13029, 200)
 +- PhotonGroupingAgg(keys=[EmpLength#13145, IntRate#13151, addr_state#13029], functions=[partial_avg(loan_amnt#13009) AS (sum#13165, count#13166L), partial_min(annual_inc#13020) AS min#13168, partial_max(annual_inc#13020) AS max#13170, partial_sum(cast(total_pymnt#13044 as double)) AS sum#13172])
 +- PhotonProject [loan_amnt#13009, annual_inc#13020, addr_state#13029, total_pymnt#13044, EmpLength#13145, IntRate#13151]
 +- PhotonBroadcastHashJoin [int_rate#13013], [int_rate#13150], LeftOuter, BuildRight, false, true
 :- PhotonProject [loan_amnt#13009, int_rate#13013, annual_inc#13020, addr_state#13029, total_pymnt#13044, EmpLength#13145]
 : +- PhotonBroadcastHashJoin [emp_length#13018], [emp_length#13144], Inner, BuildRight, false, true
 : :- PhotonProject [loan_amnt#13009, int_rate#13013, emp_length#13018, annual_inc#13020, addr_state#13029, total_pymnt#13044]
 : : +- PhotonScan parquet hive_metastore.photonperformance_mojgan_mazouchi_databricks_com_db.lendingclub_silver[loan_amnt#13009,int_rate#13013,emp_length#13018,annual_inc#13020,loan_status#13022,addr_state#13029,total_pymnt#13044] DataFilters: [isnotnull(annual_inc#13020), (annual_inc#13020 > 16000.0), isnotnull(loan_status#13022), isnotnu..., DictionaryFilters: [(annual_inc#13020 > 16000.0), (loan_status#13022 = Current)], Format: parquet, Location: PreparedDeltaFileIndex(1 paths)[dbfs:/user/mojgan.mazouchi@databricks.com/PhotonPerformance/lendi..., PartitionFilters: [], ReadSchema: struct<loan_amnt:float,int_rate:string,emp_length:string,annual_inc:float,loan_status:string,addr..., RequiredDataFilters: [isnotnull(annual_inc#13020), (annual_inc#13020 > 16000.0), isnotnull(loan_status#13022), isnotnu...
 : +- PhotonShuffleExchangeSource
 : +- PhotonShuffleMapStage
 : +- PhotonShuffleExchangeSink SinglePartition
 : +- PhotonProject [emp_length#13144, EmpLength#13145]
 : +- PhotonScan parquet hive_metastore.photonperformance_mojgan_mazouchi_databricks_com_db.lendingclub_emplength[emp_length#13144,EmpLength#13145,avg_cur_bal#13147] DataFilters: [isnotnull(avg_cur_bal#13147), (avg_cur_bal#13147 >= 1), (avg_cur_bal#13147 <= 100), isnotnull(em..., DictionaryFilters: [EmpLength#13145 IN (3-5Years,1year,Under1year), ((avg_cur_bal#13147 >= 1) AND (avg_cur_bal#13147..., Format: parquet, Location: PreparedDeltaFileIndex(1 paths)[dbfs:/user/mojgan.mazouchi@databricks.com/PhotonPerformance/lendi..., PartitionFilters: [], ReadSchema: struct<emp_length:string,EmpLength:string,avg_cur_bal:int>, RequiredDataFilters: [isnotnull(avg_cur_bal#13147), (avg_cur_bal#13147 >

### Photon-Enabled Clusters

By enabling the advice text (`set spark.databricks.adviceGenerator.acceleratedWithPhoton.enabled = true;`), you can trace photon-enabled clusters logs in the INFO section of Driver logs under Log4j output. Look specifically for "Accelerated with photon" in the logs to find out how much your queries and workloads accelerated by photon.

<img alt="Caution" title="Caution" style="vertical-align: text-bottom; position: relative; height:1.3em; top:0.0em" src="https://files.training.databricks.com/static/images/icon-warning.svg"/> 
** Advice text is disabled by default**, and you have to enable it in advance, prior to running your queries.

<img src='https://raw.githubusercontent.com/morganmazouchi/Performance-with-Photon/main/Images/log%204j%20output.png' width="2500">

In [0]:
%sql

set spark.databricks.adviceGenerator.acceleratedWithPhoton.enabled = true;

key,value
spark.databricks.adviceGenerator.acceleratedWithPhoton.enabled,true


In [0]:
%sql
SELECT
  T_len.EmpLength,
  T_rate.IntRate,
  count(DISTINCT T.addr_state) cnt_loan_by_state,
  avg(loan_amnt) avg_loan_by_state,
  min(DISTINCT annual_inc) as min_annual_income,
  max(DISTINCT annual_inc) as max_annual_income,
  sum(total_pymnt) totalPayment_by_state
FROM
  LendingClub_silver T
  LEFT JOIN 
  (SELECT row_number() OVER(PARTITION BY addr_state ORDER BY avg_cur_bal DESC) as row_num_avgBal_state, *
  FROM LendingClub_EmpLength) T_len on T_len.emp_length = T.emp_length and T_len.avg_cur_bal BETWEEN 1 AND 10
  LEFT JOIN LendingClub_IntRate T_rate on T_rate.int_rate = T.int_rate
WHERE
  (annual_inc> 16000) AND loan_status == "Current"
GROUP BY
  1,
  2
HAVING EmpLength IN ('3-5Years', '1year', 'Under1year')

EmpLength,IntRate,cnt_loan_by_state,avg_loan_by_state,min_annual_income,max_annual_income,totalPayment_by_state
1year,HighRate,50,15511.792100074785,16144.0,6000000.0,1.6845742348221095E9
Under1year,HighRate,50,15980.667546948356,16112.0,650000.0,1.687129366610441E9
3-5Years,MediumRate,50,14170.33729233472,16215.0,6200000.0,7.440206431133909E9
Under1year,MediumRate,50,14203.342335573741,16200.0,1300001.0,3.668164600199508E9
3-5Years,ExtremelyHighRate,50,17773.226341758895,16248.0,1100000.0,1.2390340980444412E9
1year,StandardRate,50,13561.58000986254,16380.0,1848400.0,3.0451219030125365E9
Under1year,ExtremelyHighRate,49,17769.116921558954,16500.0,585000.0,5.69117572940545E8
Under1year,lowRate,50,14589.907453726864,19000.0,780000.0,3.095344027199998E8
3-5Years,lowRate,50,14139.463474827246,20000.0,1574060.0,6.302428335599998E8
3-5Years,HighRate,50,16039.83564721813,16034.0,3000000.0,3.4527794636565747E9
